In [270]:
# to get html content
import requests

from requests import Session

from bs4 import BeautifulSoup
# conda install -c anaconda beautiful-soup=4.3.2
# conda install lxml

import json

### download single product

In [323]:
producturl = "http://www.sigmaaldrich.com/catalog/product/sial/41144"
    
r = requests.get(producturl)

if r.status_code == 200:
        
    soup = BeautifulSoup(r.text, "html.parser")

product infos are saved in ``` div id="productInfo" ```

In [324]:
productinfo = soup.find('div', class_='productInfo')
productinfo

<div class="productInfo twoThirds">
<strong class="strongH5" itemprop="productID"> 41144 </strong>
<strong class="strongH5" itemprop="brand"> Sigma-Aldrich </strong>
<h1 itemprop="name">
    β-Alanine
	
 



</h1>
<input id="pdpIdGA" type="hidden" value="41144">
<input id="pdpBrandGA" type="hidden" value="Sigma-Aldrich"/>
<input id="pdpNameGA" type="hidden" value="β-Alanine"/>
<h2 itemprop="description">
	analytical standard
</h2>
<!--TODO need to add country and region from cookie/profile into this -->
<p class="synonym"><span>Synonym: </span> <strong> 
		β-Ala,
		
	
		3-Aminopropionic acid
	</strong></p>
<div class="productLinks">
<a class="btn-grey" href="javascript:submitMSDSSearch('US','');" id="btn-msds">
<div class="btn-grey-left"></div>
<div class="btn-grey-mid">
					SDS
				</div>
<div class="btn-grey-right"></div>
</a>
<span class="btn-grey" id="btn-similar" itemprop="isSimilarTo">
<a class="btn-grey" href="/catalog/substance/balanine890910795911?lang=en®ion=US" id="btn-simi

In [325]:
productinfo.find(itemprop="name").get_text()

'\n    β-Alanine\n\t\n \n\n\n\n'

In [327]:
# secondary info is saved in ul list of productinfo

lielements = section.find_next('ul').find_all('li')
lielements

[<li>
 <p>CAS Number <span> <a href="/catalog/search?term=107-95-9&amp;interface=CAS No.&amp;N=0&amp;mode=partialmax&amp;lang=en®ion=US&amp;focus=product">107-95-9</a> </span></p>
 </li>, <li>
 <p>Linear Formula <span> NH<sub>2</sub>CH<sub>2</sub>CH<sub>2</sub>COOH </span></p>
 </li>, <li>
 <p>Molecular Weight <span> 89.09 </span></p>
 </li>, <li>
 <p> Beilstein Registry Number <span> 906793 </span></p>
 </li>, <li>
 <p> EC Number <span> <a href="/catalog/search?term=203-536-5&amp;interface=EG/EC No.&amp;N=0&amp;mode=partialmax&amp;lang=en®ion=US&amp;focus=product">203-536-5</a> </span></p>
 </li>, <li>
 <p> MDL number <span> <a href="/catalog/search?term=MFCD00008200&amp;interface=MDL No.&amp;N=0&amp;mode=partialmax&amp;lang=en®ion=US&amp;focus=product">MFCD00008200</a> </span></p>
 </li>, <li class="nobdr">
 <p> PubChem Substance ID <span> <a class="External" href="javascript:OpenWin('http://pubchem.ncbi.nlm.nih.gov/summary/summary.cgi?sid=329756061','height=600,width=800,scrollbars=

### define function
``` get_product_details([web page soup here]) ```

In [328]:
# get product infos

def get_product_details(soup):
    
    # initialize
    synonyms = []
    info={}
    
    # information is saved in class = "productInfo"
    # -- only price is missing (to be implemented) --
    productinfo = soup.find('div', class_='productInfo')
    
    # get primary product details
    name = productinfo.find(itemprop="name").get_text()
    name = name.replace('\t',"").replace('\n',"").replace(u'\xa0', u' ').strip()
    description = productinfo.find(itemprop="description").get_text()
    description = description.replace('\t',"").replace('\n',"").replace(u'\xa0', u' ').strip()
    pid = productinfo.find("", itemprop='productID').get_text().strip()
    brand = productinfo.find("", itemprop='brand').get_text().strip()

    synonymtext =  productinfo.find("p", class_="synonym")

    if synonymtext:

        synonymtext = synonymtext.get_text().replace('\t',"").replace('\n',"").replace(u'\xa0', u' ').strip('Synonyms:')
        synonyms = synonymtext.split(",")
        synonyms = [x.strip() for x in synonyms]
    
    price = ""
    
    info['name'] = name
    info['description'] = description
    info['pid'] = pid
    info['brand'] = brand
    info['synonyms'] = synonyms
    info['price'] = ''
    
    ## get secondary info 

    lielements = productinfo.find_next('ul').find_all('li')
    
    if lielements:
    
        keys = []
        values = []
        for li in lielements:
            value = li.find("span").get_text()
            value = value.replace('\t',"").replace('\n',"").replace(u'\xa0', u' ').strip()
            key = li.get_text().split(value)[0]
            key = key.replace('\t',"").replace('\n',"").replace(u'\xa0', u' ').strip()
    #     print(key, value)
            info[key] = value
    
    return info

info = get_product_details(soup)
info

{'Beilstein Registry Number': '906793',
 'CAS Number': '107-95-9',
 'EC Number': '203-536-5',
 'Linear Formula': 'NH2CH2CH2COOH',
 'MDL number': 'MFCD00008200',
 'Molecular Weight': '89.09',
 'PubChem Substance ID': '329756061',
 'brand': 'Sigma-Aldrich',
 'description': 'analytical standard',
 'name': 'β-Alanine',
 'pid': '41144',
 'price': '',
 'synonyms': ['β-Ala', '3-Aminopropionic acid']}

### loop over several products

In [330]:
import time

### test on couple of products throughtout their catalog

producturls = [
    "http://www.sigmaaldrich.com/catalog/product/aldrich/460591?lang=en&region=US",
    "http://www.sigmaaldrich.com/catalog/product/sial/41144",  
    "http://www.sigmaaldrich.com/catalog/product/aldrich/429422",
    "http://www.sigmaaldrich.com/catalog/product/aldrich/264806",
    "http://www.sigmaaldrich.com/catalog/product/supelco/24218u",
    "http://www.sigmaaldrich.com/catalog/product/roche/roapro",
    "http://www.sigmaaldrich.com/catalog/product/roche/10745731001",
    "http://www.sigmaaldrich.com/catalog/product/mm/xx3002564",
    "http://www.sigmaaldrich.com/catalog/product/mm/106202",
    "http://www.sigmaaldrich.com/catalog/product/sial/493538",
    "http://www.sigmaaldrich.com/catalog/product/sial/493511"
]

t = time.time()
i = 0 
for idx, producturl in enumerate(producturls): 
    
    r = requests.get(producturl)

    if r.status_code == 200:
        
        soup = BeautifulSoup(r.text, "html.parser")
        
        # get product details
        productdetails = get_product_details(soup)
        print(productdetails)
  
print("execution total", time.time() - t)
print("execution per item", (time.time() - t)/idx)

{'name': 'Sodium bromide', 'description': 'anhydrous, beads, −10 mesh, 99.999% trace metals basis', 'pid': '460591', 'brand': 'Aldrich', 'synonyms': [], 'price': '', 'CAS Number': '7647-15-6', 'Linear Formula': 'NaBr', 'Molecular Weight': '102.89', 'EC Number': '231-599-9', 'MDL number': 'MFCD00003475', 'PubChem Substance ID': '24869713', 'Popular Documents:  Specification Sheet (PDF)': 'Popular Documents:  Specification Sheet (PDF)'}
{'name': 'β-Alanine', 'description': 'analytical standard', 'pid': '41144', 'brand': 'Sigma-Aldrich', 'synonyms': ['β-Ala', '3-Aminopropionic acid'], 'price': '', 'CAS Number': '107-95-9', 'Linear Formula': 'NH2CH2CH2COOH', 'Molecular Weight': '89.09', 'Beilstein Registry Number': '906793', 'EC Number': '203-536-5', 'MDL number': 'MFCD00008200', 'PubChem Substance ID': '329756061'}
{'name': 'Potassium iodide', 'description': 'anhydrous, beads, −10 mesh, 99.998% trace metals basis', 'pid': '429422', 'brand': 'Aldrich', 'synonyms': [], 'price': '', 'CAS Num